# **PAGE RANK**



***Summarization*** adalah suatu cara untuk meringkas suatu dokumen ke dalam bentuk yang lebih sederhana dengan mengambil poin-poin penting dalam dokumen tersebut.

Secara umum, pendekatan news summarization dibagi menjadi dua, yaitu extractive dan abstractive. Extractive summarization berarti mengidentifikasi bagian-bagian penting dari teks dan membuat kalimat dari kata-kata tersebut. Sementara abstractive summarization mereproduksi materi penting dengan cara baru setelah melakukan interpretasi dan memeriksa teks menggunakan NLP untuk membuat kalimat baru yang berasal dari dokumen aslinya.




***TextRank*** adalah algoritma peringkat berbasis grafik untuk memproses teks. TextRank menghasilkan ekstraksi kalimat sebagai ringkasan. Salah satu kelebihan dari algoritma ini, tidak diperlukannya pelatihan menggunakan data training pada algoritma yang digunakan.

**Langkah pertama adalah meng-import library yang diperlukan**

In [ ]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from scipy import spatial
import networkx as nx

**Teks berita yang digunakan**

In [ ]:
text='''The Double Asteroid Redirection Test successfully changed the trajectory of the asteroid Dimorphos when the NASA spacecraft intentionally slammed into the space rock on September 26, according to the agency. The DART mission, a full-scale demonstration of deflection technology, was the world’s first conducted on behalf of planetary defense. The mission was also the first time humanity intentionally changed the motion of a celestial object in space. Prior to impact, it took Dimorphos 11 hours and 55 minutes to orbit its larger parent asteroid Didymos. Astronomers used ground-based telescopes to measure how Dimorphos’ orbit changed after impact. Now, it takes Dimorphos 11 hours and 23 minutes to circle Didymos. The DART spacecraft changed the moonlet asteroid’s orbit by 32 minutes. Initially, astronomers expected DART to be a success if it shortened the trajectory by 10 minutes. “All of us have a responsibility to protect our home planet. After all, it’s the only one we have,” said NASA Administrator Bill Nelson. “This mission shows that NASA is trying to be ready for whatever the universe throws at us. NASA has proven we are serious as a defender of the planet. This is a watershed moment for planetary defense and all of humanity, demonstrating commitment from NASA’s exceptional team and partners from around the world.”'''

In [ ]:
import nltk
nltk.download('punkt')
from nltk import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**Tokenizing paragraph ke bentuk list sentences**

In [ ]:
sentences=sent_tokenize(text)
print(sentences)

['The Double Asteroid Redirection Test successfully changed the trajectory of the asteroid Dimorphos when the NASA spacecraft intentionally slammed into the space rock on September 26, according to the agency.', 'The DART mission, a full-scale demonstration of deflection technology, was the world’s first conducted on behalf of planetary defense.', 'The mission was also the first time humanity intentionally changed the motion of a celestial object in space.', 'Prior to impact, it took Dimorphos 11 hours and 55 minutes to orbit its larger parent asteroid Didymos.', 'Astronomers used ground-based telescopes to measure how Dimorphos’ orbit changed after impact.', 'Now, it takes Dimorphos 11 hours and 23 minutes to circle Didymos.', 'The DART spacecraft changed the moonlet asteroid’s orbit by 32 minutes.', 'Initially, astronomers expected DART to be a success if it shortened the trajectory by 10 minutes.', '“All of us have a responsibility to protect our home planet.', 'After all, it’s the 

**Tahap Preprocessing**

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True



*   re.sub(r'[^\w\s]') menghapus semua tanda baca dari setiap kalimat menggunakan pemahaman daftar
*   Hapus stopwords (a, the, an, dll) untuk fokus pada kata-kata yang lebih signifikan dalam sebuah kalimat.




In [ ]:
sentences_clean=[re.sub(r'[^\w\s]','',sentence.lower()) for sentence in sentences]
stop_words = stopwords.words('english')
sentence_tokens=[[words for words in sentence.split(' ') if words not in stop_words] for sentence in sentences_clean]
print(sentence_tokens)

[['double', 'asteroid', 'redirection', 'test', 'successfully', 'changed', 'trajectory', 'asteroid', 'dimorphos', 'nasa', 'spacecraft', 'intentionally', 'slammed', 'space', 'rock', 'september', '26', 'according', 'agency'], ['dart', 'mission', 'fullscale', 'demonstration', 'deflection', 'technology', 'worlds', 'first', 'conducted', 'behalf', 'planetary', 'defense'], ['mission', 'also', 'first', 'time', 'humanity', 'intentionally', 'changed', 'motion', 'celestial', 'object', 'space'], ['prior', 'impact', 'took', 'dimorphos', '11', 'hours', '55', 'minutes', 'orbit', 'larger', 'parent', 'asteroid', 'didymos'], ['astronomers', 'used', 'groundbased', 'telescopes', 'measure', 'dimorphos', 'orbit', 'changed', 'impact'], ['takes', 'dimorphos', '11', 'hours', '23', 'minutes', 'circle', 'didymos'], ['dart', 'spacecraft', 'changed', 'moonlet', 'asteroids', 'orbit', '32', 'minutes'], ['initially', 'astronomers', 'expected', 'dart', 'success', 'shortened', 'trajectory', '10', 'minutes'], ['us', 'res

Seperti yang kita ketahui kita perlu mengkonversi data teks ke dalam bentuk numerik, pertama kita akan menghitung penyisipan kata & menggunakan penyematan ini akan menghitung penyisipan kalimat menggunakan variabel sentence_tokens yang dihitung di atas.

- Menggunakan Word2Vec() gensim, menghitung embedding untuk setiap kata dari teks.
- Menggunakan pemahaman daftar, kami akan mengganti token kata dengan penyematannya di kalimat_tokens
- Menghitung max_length kalimat dalam teks yang ada
- Penyematan kalimat padding menggunakan 0s hingga max_len. Ini telah dilakukan untuk mempertahankan ukuran embedding yang sama untuk setiap kalimat.

In [ ]:
w2v=Word2Vec(sentence_tokens,size=1,min_count=1,iter=1000)
sentence_embeddings=[[w2v[word][0] for word in words] for words in sentence_tokens]
max_len=max([len(tokens) for tokens in sentence_tokens])
sentence_embeddings=[np.pad(embedding,(0,max_len-len(embedding)),'constant') for embedding in sentence_embeddings]
print(sentence_embeddings)

[array([-1.3774105, -1.5648252, -1.4396875, -1.4545798, -1.4218374,
       -1.5217278, -1.5086281, -1.5648252, -1.5196548, -1.6093941,
       -1.4879727, -1.5637304, -1.4247156, -1.6206998, -1.5261114,
       -1.6321287, -1.6664679, -1.6027577, -1.6043236], dtype=float32), array([-1.58468  , -1.6650534, -1.5069708, -1.683015 , -1.6686802,
       -1.7436057, -1.6813625, -1.6704615, -1.6362563, -1.7113441,
       -1.803447 , -1.7605822,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ], dtype=float32), array([-1.6650534, -1.4216933, -1.6704615, -1.4764392, -1.6757175,
       -1.5637304, -1.5217278, -1.4476883, -1.4730337, -1.5155343,
       -1.6206998,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ], dtype=float32), array([-1.3958535, -1.4649928, -1.4221333, -1.5196548, -1.5319772,
       -1.5061108, -1.3955861, -1.492912 , -1.4730704, -1.392894 ,
       -1.340811 , -1.5648252, -1.5237055,  0.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


**calculate the similarity matrix**


> Inisialisasi kesamaan_matriks dimensi N x N di mana N adalah jumlah total kalimat dalam teks

> Menggunakan 1-spatial.distance.cosine(), hitung kesamaan antara setiap dua pasang kalimat.





In [ ]:
similarity_matrix = np.zeros([len(sentence_tokens), len(sentence_tokens)])
for i,row_embedding in enumerate(sentence_embeddings):
    for j,column_embedding in enumerate(sentence_embeddings):
        similarity_matrix[i][j]=1-spatial.distance.cosine(row_embedding,column_embedding)
print(similarity_matrix)

[[1.         0.77842277 0.73990798 0.80623448 0.66596884 0.62674886
  0.6247347  0.66628879 0.48455659 0.53557593 0.66616797 0.48269325
  0.8068682 ]
 [0.77842277 1.         0.94986784 0.95573455 0.85033149 0.80226189
  0.79970229 0.85097426 0.62316644 0.69113314 0.85086727 0.62143332
  0.96138918]
 [0.73990798 0.94986784 1.         0.90760857 0.89940935 0.85212719
  0.85283911 0.89906245 0.68671745 0.7514078  0.90093911 0.68750477
  0.91812694]
 [0.80623448 0.95573455 0.90760857 1.         0.83176255 0.78460878
  0.78243357 0.83306611 0.61989808 0.68305069 0.83296162 0.61842948
  0.99759716]
 [0.66596884 0.85033149 0.89940935 0.83176255 1.         0.94004345
  0.94033813 0.99717361 0.73086035 0.80639416 0.99925023 0.72986954
  0.81961304]
 [0.62674886 0.80226189 0.85212719 0.78460878 0.94004345 1.
  0.99711019 0.94154871 0.78555882 0.86498463 0.94156569 0.78253978
  0.77424937]
 [0.6247347  0.79970229 0.85283911 0.78243357 0.94033813 0.99711019
  1.         0.93804789 0.79763031 0.874

**implement PageRank**


> Convert the similarity matrix to a network/graph


> Apply pagerank()





In [ ]:
nx_graph = nx.from_numpy_array(similarity_matrix)
scores = nx.pagerank(nx_graph)
print(scores)

{0: 0.06648528095912415, 1: 0.07784784411812283, 2: 0.07975563663364352, 3: 0.07735306264708693, 4: 0.08075394198654515, 5: 0.08003775721426168, 6: 0.0801811591515655, 7: 0.08108974282846147, 8: 0.07171796050837635, 9: 0.07537486763119604, 10: 0.08093280912219844, 11: 0.07162080061892813, 12: 0.07684913658048982}


Sekarang, yang akan kita lakukan adalah membuat kamus dengan kalimat (kita dapatkan setelah sent_tokenize()) sebagai kunci & PageRank mereka sebagai nilai & mengurutkan kamus ini berdasarkan PageRanks kalimat. Disini diambil 4 kalimat teratas untuk ringkasan:

In [ ]:
top_sentence={sentence:scores[index] for index,sentence in enumerate(sentences)}
top=dict(sorted(top_sentence.items(), key=lambda x: x[1], reverse=True)[:4])
print(top)

{'Initially, astronomers expected DART to be a success if it shortened the trajectory by 10 minutes.': 0.08108974282846147, '“This mission shows that NASA is trying to be ready for whatever the universe throws at us.': 0.08093280912219844, 'Astronomers used ground-based telescopes to measure how Dimorphos’ orbit changed after impact.': 0.08075394198654515, 'The DART spacecraft changed the moonlet asteroid’s orbit by 32 minutes.': 0.0801811591515655}


In [ ]:
for sent in sentences:
    if sent in top.keys():
        print(sent)

Astronomers used ground-based telescopes to measure how Dimorphos’ orbit changed after impact.
The DART spacecraft changed the moonlet asteroid’s orbit by 32 minutes.
Initially, astronomers expected DART to be a success if it shortened the trajectory by 10 minutes.
“This mission shows that NASA is trying to be ready for whatever the universe throws at us.


# **SENTIMEN ANALISIS**

# **Data crawling**
Data crawling atau perayapan data adalah proses pengambilan data yang tersedia secara online untuk umum. Proses ini kemudian mengimpor informasi atau data yang telah ditemukan ke dalam file lokal di komputer Anda. 

## Twint

Twint adalah sebuah tools yang digunakan untuk melakukan scrapping dari aplikasi twitter yang disetting secara khusus menggunakan bahasa pemrograman Python. Twint dapat kita gunakan dan jalankan tanpa harus menggunakan API dari Twitter itu sendiri, dengan kapasitas scrapping data maksimalnya adalah 3200 tweet.

Bukan hanya digunakan pada tweet, twint juga bisa kita gunakan untuk melakukan scrapping pada user, followers, retweet dan sebagainya. Twint memanfaatkan operator pencarian twitter untuk memungkinkan proses penghapusan tweet dari user tertentu, memilih dan memilah informasi-informasi yang sensitif, termasuk email dan nomor telepon di dalamnya.

In [ ]:
!git clone --depth=1 https://github.com/twintproject/twint.git
%cd twint
!pip3 install . -r requirements.txt

Cloning into 'twint'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 47 (delta 3), reused 14 (delta 0), pack-reused 0
Unpacking objects: 100% (47/47), done.
/content/twint
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/twint
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 263 kB 2.1 MB/s 
     |████████████████████████████████| 384 kB 35.8 MB/s 
     |████████████████████████████████| 288 kB 58.8 MB/s 
     |█████████████████████████

In [ ]:
!pip install nest-asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install aiohttp==3.7.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 2.1 MB/s 
  Attempting uninstall: async-timeout
    Found existing installation: async-timeout 4.0.2
    Uninstalling async-timeout-4.0.2:
      Successfully uninstalled async-timeout-4.0.2
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.8.3
    Uninstalling aiohttp-3.8.3:
      Successfully uninstalled aiohttp-3.8.3


In [ ]:
import nest_asyncio
nest_asyncio.apply() #digunakan sekali untuk mengaktifkan tindakan serentak dalam notebook jupyter.
import twint #untuk import twint
c = twint.Config()
c.Search = 'tragedi kanjuruhan'
c.Pandas = True
c.Limit = 70
twint.run.Search(c)

1579997596319305728 2022-10-12 00:49:16 +0000 <roudytoop> Polri menegaskan gas air mata, termasuk yang digunakan anggotanya dalam tragedi Kanjuruhan, tidak mematikan. Polri menjelaskan pendapat itu dikeluarkan oleh pakar-pakar racun dan gas air mata.  https://t.co/LfkMZCf8NX.  https://t.co/fCdEccXYCi
1579997543840509952 2022-10-12 00:49:04 +0000 <MafiaWasit> Disaat kita sedang fokus #USUTTUNTAS Tragedi Kanjuruhan, bahkan Pak Jokowi saja sampai ikut turun tangan menyelesaikan sistem bobrok persepakbolaan Indonesia! Lu malah bahas UCL?  NURANI LU GA ADA BABAR BLAS..   TAK DOAKAN KOWE ANYANG-ANYANGEN SEMINGGU!  https://t.co/WD20hZXpLI
1579997523531399168 2022-10-12 00:48:59 +0000 <PortalSDAdotcom> Polisi Tak Menahan Tersangka Tragedi Kanjuruhan Malang  https://t.co/I3Nf7QImVa
1579997284292788224 2022-10-12 00:48:02 +0000 <RMOLJatim_id> Soal Gas Air Mata Kadaluarsa Tragedi Kanjuruhan, Mahfud MD: Sedang Diperiksa di Laboratorium: Polri telah mengakui kalau gas air mata yang dipakai saat men

In [ ]:
Tweets_dfs = twint.storage.panda.Tweets_df
Tweets_dfs["tweet"]

0     Polri menegaskan gas air mata, termasuk yang d...
1     Disaat kita sedang fokus #USUTTUNTAS Tragedi K...
2     Polisi Tak Menahan Tersangka Tragedi Kanjuruha...
3     Soal Gas Air Mata Kadaluarsa Tragedi Kanjuruha...
4     Panpel Arema FC Terang-terangan Sebut Nama Ket...
                            ...                        
75    Gas Air Mata Sebab Tragedi Kanjuruhan, Said Di...
76    Ade Armando dipolisikan terkait Tragedi Kanjur...
77    11 HARI DALAM PERAWATAN. sebuah catatan buat t...
78    Polisi dalam Tragedi Kanjuruhan: Sujud Minta M...
79    Aamiin, Aamiin, aamiin ya Allah ya rabbal alla...
Name: tweet, Length: 80, dtype: object

In [ ]:
Tweets_dfs["tweet"].to_csv("TRAGEDI.csv")

# **Mengambil Data**

Proses ini digunakan untuk mengambil 100 data tweet yang telah disimpan dalam github dengan format .csv



In [ ]:
#install library pandas
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#install library numpy
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd 
import numpy as np

data_abstrak = pd.read_csv("TRAGEDI.csv")

data_abstrak

,Unnamed: 0,tweet
0,0,"Polri menegaskan gas air mata, termasuk yang d..."
1,1,Disaat kita sedang fokus #USUTTUNTAS Tragedi K...
2,2,Polisi Tak Menahan Tersangka Tragedi Kanjuruha...
3,3,Soal Gas Air Mata Kadaluarsa Tragedi Kanjuruha...
4,4,Panpel Arema FC Terang-terangan Sebut Nama Ket...
...,...,...
75,75,"Gas Air Mata Sebab Tragedi Kanjuruhan, Said Di..."
76,76,Ade Armando dipolisikan terkait Tragedi Kanjur...
77,77,11 HARI DALAM PERAWATAN. sebuah catatan buat t...
78,78,Polisi dalam Tragedi Kanjuruhan: Sujud Minta M...


In [ ]:
#install library sastrawi
!pip install sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 2.1 MB/s 


In [ ]:
#install library swifter
!pip install swifter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 830 kB 2.1 MB/s 
     |████████████████████████████████| 281 kB 54.9 MB/s 
     |████████████████████████████████| 1.6 MB 53.0 MB/s 
  Created wheel for swifter: filename=swifter-1.3.4-py3-none-any.whl size=16322 sha256=1b53537bfbcc2dcbe3a0b3db06193f3d85a4a9d2592a6e10908ea0c905d2eb10
  Stored in directory: /root/.cache/pip/wheels/29/a7/0e/3a8f17ac69d759e1e93647114bc9bdc95957e5b0cbfd405205
Successfully built swifter
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


# *Case Folding**

Tahap untuk merubah teks yang memiliki huruf kapital menjadi huruf kecil

In [ ]:
data_abstrak['tweet'] = data_abstrak['tweet'].str.lower()


data_abstrak['tweet']

0     polri menegaskan gas air mata, termasuk yang d...
1     disaat kita sedang fokus #usuttuntas tragedi k...
2     polisi tak menahan tersangka tragedi kanjuruha...
3     soal gas air mata kadaluarsa tragedi kanjuruha...
4     panpel arema fc terang-terangan sebut nama ket...
                            ...                        
75    gas air mata sebab tragedi kanjuruhan, said di...
76    ade armando dipolisikan terkait tragedi kanjur...
77    11 hari dalam perawatan. sebuah catatan buat t...
78    polisi dalam tragedi kanjuruhan: sujud minta m...
79    aamiin, aamiin, aamiin ya allah ya rabbal alla...
Name: tweet, Length: 80, dtype: object

In [ ]:
#install library nltk
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Menghapus Karakter Spesial

In [ ]:
import string 
import re #regex library
# import word_tokenize & FreqDist from NLTK

from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

# ------ Tokenizing ---------

def remove_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\'," ").replace('\\f'," ").replace('\\r'," ")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
data_abstrak['tweet'] = data_abstrak['tweet'].apply(remove_special)
data_abstrak['tweet']


0     polri menegaskan gas air mata, termasuk yang d...
1     disaat kita sedang fokus tragedi kanjuruhan, b...
2     polisi tak menahan tersangka tragedi kanjuruha...
3     soal gas air mata kadaluarsa tragedi kanjuruha...
4     panpel arema fc terang-terangan sebut nama ket...
                            ...                        
75    gas air mata sebab tragedi kanjuruhan, said di...
76    ade armando dipolisikan terkait tragedi kanjur...
77    11 hari dalam perawatan. sebuah catatan buat t...
78    polisi dalam tragedi kanjuruhan: sujud minta m...
79    aamiin, aamiin, aamiin ya allah ya rabbal alla...
Name: tweet, Length: 80, dtype: object

## Menghapus Angka

In [ ]:

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

data_abstrak['tweet'] = data_abstrak['tweet'].apply(remove_number)
data_abstrak['tweet']

0     polri menegaskan gas air mata, termasuk yang d...
1     disaat kita sedang fokus tragedi kanjuruhan, b...
2     polisi tak menahan tersangka tragedi kanjuruha...
3     soal gas air mata kadaluarsa tragedi kanjuruha...
4     panpel arema fc terang-terangan sebut nama ket...
                            ...                        
75    gas air mata sebab tragedi kanjuruhan, said di...
76    ade armando dipolisikan terkait tragedi kanjur...
77     hari dalam perawatan. sebuah catatan buat tem...
78    polisi dalam tragedi kanjuruhan: sujud minta m...
79    aamiin, aamiin, aamiin ya allah ya rabbal alla...
Name: tweet, Length: 80, dtype: object

## Menghapus Tanda Baca

In [ ]:

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

data_abstrak['tweet'] = data_abstrak['tweet'].apply(remove_punctuation)
data_abstrak['tweet']

0     polri menegaskan gas air mata termasuk yang di...
1     disaat kita sedang fokus tragedi kanjuruhan ba...
2     polisi tak menahan tersangka tragedi kanjuruha...
3     soal gas air mata kadaluarsa tragedi kanjuruha...
4     panpel arema fc terangterangan sebut nama ketu...
                            ...                        
75    gas air mata sebab tragedi kanjuruhan said did...
76    ade armando dipolisikan terkait tragedi kanjur...
77     hari dalam perawatan sebuah catatan buat tema...
78    polisi dalam tragedi kanjuruhan sujud minta ma...
79    aamiin aamiin aamiin ya allah ya rabbal allami...
Name: tweet, Length: 80, dtype: object

## Menghapus Spasi


In [ ]:

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

data_abstrak['tweet'] = data_abstrak['tweet'].apply(remove_whitespace_LT)


#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

data_abstrak['tweet'] = data_abstrak['tweet'].apply(remove_whitespace_multiple)
data_abstrak['tweet']

0     polri menegaskan gas air mata termasuk yang di...
1     disaat kita sedang fokus tragedi kanjuruhan ba...
2     polisi tak menahan tersangka tragedi kanjuruha...
3     soal gas air mata kadaluarsa tragedi kanjuruha...
4     panpel arema fc terangterangan sebut nama ketu...
                            ...                        
75    gas air mata sebab tragedi kanjuruhan said did...
76    ade armando dipolisikan terkait tragedi kanjur...
77    hari dalam perawatan sebuah catatan buat teman...
78    polisi dalam tragedi kanjuruhan sujud minta ma...
79    aamiin aamiin aamiin ya allah ya rabbal allami...
Name: tweet, Length: 80, dtype: object

## Menghapus huruf 

In [ ]:

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", " ", text)

data_abstrak['tweet'] = data_abstrak['tweet'].apply(remove_singl_char)
data_abstrak['tweet']

0     polri menegaskan gas air mata termasuk yang di...
1     disaat kita sedang fokus tragedi kanjuruhan ba...
2     polisi tak menahan tersangka tragedi kanjuruha...
3     soal gas air mata kadaluarsa tragedi kanjuruha...
4     panpel arema fc terangterangan sebut nama ketu...
                            ...                        
75    gas air mata sebab tragedi kanjuruhan said did...
76    ade armando dipolisikan terkait tragedi kanjur...
77    hari dalam perawatan sebuah catatan buat teman...
78    polisi dalam tragedi kanjuruhan sujud minta ma...
79    aamiin aamiin aamiin ya allah ya rabbal allami...
Name: tweet, Length: 80, dtype: object

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# **Tokenizing**

Tokenizing adalah proses pemisahan teks menjadi potongan-potongan yang disebut sebagai token untuk kemudian di analisa. Kata, angka, simbol, tanda baca dan entitas penting lainnya dapat dianggap sebagai token

In [ ]:

# NLTK word Tokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

data_abstrak['tweet'] = data_abstrak['tweet'].apply(word_tokenize_wrapper)
data_abstrak['tweet']

0     [polri, menegaskan, gas, air, mata, termasuk, ...
1     [disaat, kita, sedang, fokus, tragedi, kanjuru...
2     [polisi, tak, menahan, tersangka, tragedi, kan...
3     [soal, gas, air, mata, kadaluarsa, tragedi, ka...
4     [panpel, arema, fc, terangterangan, sebut, nam...
                            ...                        
75    [gas, air, mata, sebab, tragedi, kanjuruhan, s...
76    [ade, armando, dipolisikan, terkait, tragedi, ...
77    [hari, dalam, perawatan, sebuah, catatan, buat...
78    [polisi, dalam, tragedi, kanjuruhan, sujud, mi...
79    [aamiin, aamiin, aamiin, ya, allah, ya, rabbal...
Name: tweet, Length: 80, dtype: object

# **Filtering(Stopwords Removal)**
Proses untuk menghapus kata hubung atau kata yang tidak memiliki makna


In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:

list_stopwords = stopwords.words('indonesian')

#Menghapus Stopword dari list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

data_abstrak['tweet'] = data_abstrak['tweet'].apply(stopwords_removal)

data_abstrak['tweet']

0     [polri, gas, air, mata, anggotanya, tragedi, k...
1     [disaat, fokus, tragedi, kanjuruhan, jokowi, t...
2     [polisi, menahan, tersangka, tragedi, kanjuruh...
3     [gas, air, mata, kadaluarsa, tragedi, kanjuruh...
4     [panpel, arema, fc, terangterangan, nama, ketu...
                            ...                        
75    [gas, air, mata, tragedi, kanjuruhan, said, di...
76    [ade, armando, dipolisikan, terkait, tragedi, ...
77    [perawatan, catatan, teman, teman, penyintas, ...
78    [polisi, tragedi, kanjuruhan, sujud, maaf, mal...
79    [aamiin, aamiin, aamiin, ya, allah, ya, rabbal...
Name: tweet, Length: 80, dtype: object

# **Stemming**
Data training hasil dari filtering akan dilakukan pengecekan atau pencarian kata-kata yang sesuai dengan kamus umum. Apabila data training hasil filtering sesuai dengan kamus umum maka kata akan dikeluarkan sementara, karena sudah dianggap sebagai kata dasar. Apabila masih terdapat kata yang tidak termasuk dalam kata dasar maka tahap selanjutnya adalah menghapus inflection suffixes yang merupakan akhiran pertama. Kata yang memiliki akhiran partticles seperti “-pun”, “-kah”, “-tah”, “- lah” dan akhiran possessive pronoun seperti “-mu”, “-ku” dan “-nya” dihilangkan. Setelah dilakukan proses case folding, tokenezing, dan filtering, proses selanjutnya yaitu stemming. Stemming yang digunakan pada penelitian ini menggunakan algoritma Enhanced Confix Stipping Stemmer, terdiri dari beberapa langkah: Data training hasil dari filtering akan dilakukan pengecekan atau pencarian kata-kata yang sesuai dengan kamus umum. Apabila data training hasil filtering sesuai dengan kamus umum maka kata akan dikeluarkan sementara, karena sudah dianggap sebagai kata dasar. Apabila masih terdapat kata yang tidak termasuk dalam kata dasar maka tahap selanjutnya adalah menghapus inflection suffixes yang merupakan akhiran pertama. Kata yang memiliki akhiran partticles seperti “-pun”, “-kah”, “-tah”, “- lah” dan akhiran possessive pronoun seperti “-mu”, “-ku” dan “-nya” dihilangkan.

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter


# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in data_abstrak['tweet']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

data_abstrak['tweet'] = data_abstrak['tweet'].swifter.apply(get_stemmed_term)
data_abstrak['tweet']

508
------------------------
polri : polri
gas : gas
air : air
mata : mata
anggotanya : anggota
tragedi : tragedi
kanjuruhan : kanjuruhan
mematikan : mati
pendapat : dapat
dikeluarkan : keluar
pakarpakar : pakarpakar
racun : racun
disaat : saat
fokus : fokus
jokowi : jokowi
turun : turun
tangan : tangan
menyelesaikan : selesai
sistem : sistem
bobrok : bobrok
persepakbolaan : persepakbolaan
indonesia : indonesia
lu : lu
bahas : bahas
ucl : ucl
nurani : nurani
ga : ga
babar : babar
blas : blas
doakan : doa
kowe : kowe
anyanganyangen : anyanganyangen
seminggu : minggu
polisi : polisi
menahan : tahan
tersangka : sangka
kanjuruhanmalang : kanjuruhanmalang
kadaluarsa : kadaluarsa
mahfud : mahfud
md : md
diperiksa : periksa
laboratorium : laboratorium
mengakui : aku
dipakai : pakai
mengamankan : aman
laga : laga
arema : arema
vs : vs
persebaya : baya
stadion : stadion
malang : malang
baca : baca
selengkapnya : lengkap
panpel : panpel
fc : fc
terangterangan : terangterangan
nama : nama
ketua :

Pandas Apply:   0%|          | 0/80 [00:00<?, ?it/s]

0     [polri, gas, air, mata, anggota, tragedi, kanj...
1     [saat, fokus, tragedi, kanjuruhan, jokowi, tur...
2     [polisi, tahan, sangka, tragedi, kanjuruhanmal...
3     [gas, air, mata, kadaluarsa, tragedi, kanjuruh...
4     [panpel, arema, fc, terangterangan, nama, ketu...
                            ...                        
75    [gas, air, mata, tragedi, kanjuruhan, said, di...
76    [ade, armando, polisi, kait, tragedi, kanjuruhan]
77    [awat, catat, teman, teman, sintas, tragedi, k...
78    [polisi, tragedi, kanjuruhan, sujud, maaf, mal...
79    [aamiin, aamiin, aamiin, ya, allah, ya, rabbal...
Name: tweet, Length: 80, dtype: object

##Menyimpan Hasil Tahap Preprocessing ke file .csv

In [ ]:

data_abstrak.to_csv('preprocessing.csv')

# **TF**

TF(Term Frequency) : Istilah frekuensi kata dalam dokumen. Ada beberapa cara untuk menghitung frekuensi ini, dengan cara yang paling sederhana adalah dengan menghitung jumlah kata yang muncul dalam dokumen. Lalu, ada cara untuk menyesuaikan frekuensi, berdasarkan panjang dokumen, atau dengan frekuensi mentah kata yang paling sering muncul dalam dokumen.

In [ ]:

from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer
#Membuat Dataframe
dataTextPre = pd.read_csv('preprocessing.csv')
vectorizer = CountVectorizer(min_df=1)
bag = vectorizer.fit_transform(dataTextPre['tweet'])
dataTextPre

,Unnamed: 0,Unnamed: 0.1,tweet
0,0,0,"['polri', 'gas', 'air', 'mata', 'anggota', 'tr..."
1,1,1,"['saat', 'fokus', 'tragedi', 'kanjuruhan', 'jo..."
2,2,2,"['polisi', 'tahan', 'sangka', 'tragedi', 'kanj..."
3,3,3,"['gas', 'air', 'mata', 'kadaluarsa', 'tragedi'..."
4,4,4,"['panpel', 'arema', 'fc', 'terangterangan', 'n..."
...,...,...,...
75,75,75,"['gas', 'air', 'mata', 'tragedi', 'kanjuruhan'..."
76,76,76,"['ade', 'armando', 'polisi', 'kait', 'tragedi'..."
77,77,77,"['awat', 'catat', 'teman', 'teman', 'sintas', ..."
78,78,78,"['polisi', 'tragedi', 'kanjuruhan', 'sujud', '..."


In [ ]:
matrik_vsm=bag.toarray()
#print(matrik_vsm)
matrik_vsm.shape

(80, 473)

In [ ]:
matrik_vsm[0]

array([0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,

In [ ]:
a=vectorizer.get_feature_names()

In [ ]:
print(len(matrik_vsm[:,1]))
#dfb =pd.DataFrame(data=matrik_vsm,index=df,columns=[a])
dataTF =pd.DataFrame(data=matrik_vsm,index=list(range(1, len(matrik_vsm[:,1])+1, )),columns=[a])
dataTF

80


,aamiin,abang,acara,ade,afinta,air,aja,ajak,ajar,akbp,...,via,video,viral,vs,wanita,ws,ya,yg,zainul,zurich
1,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,2,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
77,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
78,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
79,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
